In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing
import tensorflow as tf

In [2]:
data_train = pd.read_csv('../train_data.csv').values
data_test = pd.read_csv('../test_data.csv').values

# Build X and y
X_train = data_train[:, 1:]
y_train = data_train[:, 0]
X_test = data_test[:, 1:]
y_test = data_test[:, 0]

# Number of stocks in training data
n_att = X_train.shape[1]

# Neurons
n_neurons_1 = 1024
n_neurons_2 = 512
n_neurons_3 = 256
n_neurons_4 = 128

# Placeholder
X = tf.placeholder(dtype=tf.float32, shape=[None, n_att])
Y = tf.placeholder(dtype=tf.float32, shape=[None])

# Initializers
sigma = 1
weight_initializer = tf.variance_scaling_initializer(mode="fan_avg", distribution="uniform", scale=sigma)
bias_initializer = tf.zeros_initializer()

# Hidden weights
W_hidden_1 = tf.Variable(weight_initializer([n_att, n_neurons_1]))
bias_hidden_1 = tf.Variable(bias_initializer([n_neurons_1]))
W_hidden_2 = tf.Variable(weight_initializer([n_neurons_1, n_neurons_2]))
bias_hidden_2 = tf.Variable(bias_initializer([n_neurons_2]))
W_hidden_3 = tf.Variable(weight_initializer([n_neurons_2, n_neurons_3]))
bias_hidden_3 = tf.Variable(bias_initializer([n_neurons_3]))
W_hidden_4 = tf.Variable(weight_initializer([n_neurons_3, n_neurons_4]))
bias_hidden_4 = tf.Variable(bias_initializer([n_neurons_4]))

# Output weights
W_out = tf.Variable(weight_initializer([n_neurons_4, 1]))
bias_out = tf.Variable(bias_initializer([1]))

# Hidden layer
hidden_1 = tf.nn.relu(tf.add(tf.matmul(X, W_hidden_1), bias_hidden_1))
hidden_2 = tf.nn.relu(tf.add(tf.matmul(hidden_1, W_hidden_2), bias_hidden_2))
hidden_3 = tf.nn.relu(tf.add(tf.matmul(hidden_2, W_hidden_3), bias_hidden_3))
hidden_4 = tf.nn.relu(tf.add(tf.matmul(hidden_3, W_hidden_4), bias_hidden_4))

# Output layer (transpose!)
out = tf.transpose(tf.add(tf.matmul(hidden_4, W_out), bias_out))

# Cost function
mse = tf.reduce_mean(tf.squared_difference(out, Y))

# Optimizer
opt = tf.train.AdamOptimizer().minimize(mse)

# Saver
saver = tf.train.Saver()

In [3]:
with tf.Session() as sess:

    # Init
    sess.run(tf.global_variables_initializer())

    # Fit neural net
    batch_size = 256
    mse_train = []
    mse_test = []

    # Run
    epochs = 15
    for e in range(epochs):

        # Shuffle training data
        shuffle_indices = np.random.permutation(np.arange(len(y_train)))
        X_train = X_train[shuffle_indices]
        y_train = y_train[shuffle_indices]

        # Minibatch training
        for i in range(0, len(y_train) // batch_size):
            start = i * batch_size
            batch_x = X_train[start:start + batch_size]
            batch_y = y_train[start:start + batch_size]
            # Run optimizer with batch
            sess.run(opt, feed_dict={X: batch_x, Y: batch_y})

            # Show progress
            if np.mod(i, 50) == 0:
                # MSE train and test
                mse_train.append(sess.run(mse, feed_dict={X: X_train, Y: y_train}))
                mse_test.append(sess.run(mse, feed_dict={X: X_test, Y: y_test}))
                print('MSE Train: ', mse_train[-1])
                print('MSE Test: ', mse_test[-1])
                # Prediction
                pred = sess.run(out, feed_dict={X: X_test})
    save_path = saver.save(sess, "../models/model.ckpt")
    print("Model saved in path: %s" % save_path)

MSE Train:  0.6876732
MSE Test:  0.6879796
MSE Train:  0.0013114874
MSE Test:  0.001313986
MSE Train:  0.0002839856
MSE Test:  0.00029852343
MSE Train:  0.00018307405
MSE Test:  0.00020047327
MSE Train:  0.00012580902
MSE Test:  0.00014684688
MSE Train:  0.00010500779
MSE Test:  0.00012624335
MSE Train:  8.632836e-05
MSE Test:  0.00010843943
MSE Train:  8.7864486e-05
MSE Test:  0.000112909336
MSE Train:  9.427103e-05
MSE Test:  0.00010981417
MSE Train:  5.120319e-05
MSE Test:  7.096556e-05
MSE Train:  5.14319e-05
MSE Test:  7.22867e-05
MSE Train:  6.2807856e-05
MSE Test:  8.459543e-05
MSE Train:  3.588693e-05
MSE Test:  5.61574e-05
MSE Train:  3.072135e-05
MSE Test:  5.1438252e-05
MSE Train:  3.142718e-05
MSE Test:  5.1363793e-05
Model saved in path: ../models/model.ckpt
